In [ ]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

FINAL_DATA_NAME = 'approx1e5-agriVision-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "agriVision-RGB-cleaned"

data_dir = os.path.join(ROOT_DIR, 'raw-data','agriVision')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir

In [ ]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))

In [ ]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

In [ ]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

In [ ]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

In [ ]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))
BATCH_NUM = 0

# Fourier

In [ ]:
#Values obtained from plots in agriVisionFourierEDA.ipynb
batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(0.008052940675034493,0.5282905274067067, 1.15)
print(splits)

In [ ]:
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))
channel_fr = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

In [ ]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

In [ ]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

In [ ]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

In [ ]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "infrared"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

In [ ]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [ ]:
FINAL_DATA_NAME = 'approx1e5-agriVision-wavelet'
batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [ ]:
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0]).astype(int)
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"test_int_batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
channel = "infrared"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float16))
pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

In [ ]:
del channel_wv, group_data_map, group_data_map_size